In [30]:
import sys
import rospy
import moveit_commander
import copy
import moveit_msgs.msg
import geometry_msgs.msg
from math import pi
from tf.transformations import *
from moveit_msgs.msg import RobotState, Constraints, JointConstraint, OrientationConstraint

In [2]:

rospy.init_node('test_ur', anonymous=True)
moveit_commander.roscpp_initialize(sys.argv)

robot = moveit_commander.RobotCommander()
scene = moveit_commander.PlanningSceneInterface()

robot.get_group_names()

['arm', 'effector']

In [3]:
group_name = robot.get_group_names()
move_group = moveit_commander.MoveGroupCommander(group_name[0])

In [4]:
planning_frame = move_group.get_planning_frame()

In [5]:
planning_frame

'world'

In [6]:
eef_link = move_group.get_end_effector_link()

In [7]:
eef_link

'ee_link'

In [8]:

robot.get_current_state()


joint_state: 
  header: 
    seq: 0
    stamp: 
      secs: 0
      nsecs:         0
    frame_id: "world"
  name: [shoulder_pan_joint, shoulder_lift_joint, elbow_joint, wrist_1_joint, wrist_2_joint,
  wrist_3_joint]
  position: [-0.857881372732769, -1.3719816527987465, 0.9830923985731443, 0.38876961109532715, -0.8585519058597715, 0.016405533281314888]
  velocity: []
  effort: []
multi_dof_joint_state: 
  header: 
    seq: 0
    stamp: 
      secs: 0
      nsecs:         0
    frame_id: "world"
  joint_names: []
  transforms: []
  twist: []
  wrench: []
attached_collision_objects: []
is_diff: False

In [9]:

move_group.set_named_target("up")
plan = move_group.plan()

In [10]:
move_group.execute(plan, wait=True)

True

In [11]:

def move_joints(move_group, goal):
    move_group.go(goal, wait=True)
    move_group.stop()
    
def get_joint_state(move_group):
    joint_state = move_group.get_current_joint_values()
    print "---- rad value ----"
    print joint_state
    print "---- degree value ---- "
    print [joint*180/math.pi for joint in joint_state]

In [12]:

import math

get_joint_state(move_group)

---- rad value ----
[0.05202944790448871, -1.6835228126736155, 0.39928326412308124, -0.6075882567708888, -1.5447613514937952, 1.822433618946084]
---- degree value ---- 
[2.9810677753229884, -96.45875188019167, 22.877245864459876, -34.812242794682895, -88.50830579551956, 104.41775480836351]


In [13]:
goal = [ n * math.pi / 180 for n in [90,-100,90,-100,-90,0]]

move_joints(move_group, goal)
get_joint_state(move_group)

---- rad value ----
[1.570700040999938, -1.7453889801837432, 1.5708907961708753, -1.745273560869733, -1.57070716698858, -4.064910985235315e-05]
---- degree value ---- 
[89.99448323032182, -100.00342217317136, 90.00541269653681, -99.99680913360427, -89.99489151939586, -0.0023290224355034883]


In [14]:
# cartesin value retrieval

def get_goal_pose(move_group):
    joint_state = move_group.get_current_pose()
    print joint_state

In [15]:

get_goal_pose(move_group)

header: 
  seq: 0
  stamp: 
    secs: 437
    nsecs: 155000000
  frame_id: "world"
pose: 
  position: 
    x: -0.109115977414
    y: 0.429564338847
    z: 1.5288228794
  orientation: 
    x: -0.405580139188
    y: 0.405631812675
    z: 0.579215806671
    w: 0.579203446593


In [16]:

import geometry_msgs.msg

pose_goal = geometry_msgs.msg.Pose()

pose_goal.position.x = 0.562
pose_goal.position.y = 0.550
pose_goal.position.z = 1.37679003904

pose_goal.orientation.x = 0.768049136044
pose_goal.orientation.y = -0.00683168224924
pose_goal.orientation.z = 0.00497378503076
pose_goal.orientation.w = 0.640335157712
move_group.set_pose_target(pose_goal)

아래 있는 cell 이 quaternion orientation 이 제대로 잡혀 있음... 여기에서 contraint 를 한번 걸어보는 것으로 해보기 ... 

In [42]:
pose_goal.position.x = 0.362
pose_goal.position.y = 0.0
pose_goal.position.z = 1.37679003904 - 0.376

pose_goal.orientation.x = 0.768049136044
pose_goal.orientation.y = -0.00683168224924
pose_goal.orientation.z = 0.00497378503076
pose_goal.orientation.w = 0.640335157712
move_group.set_pose_target(pose_goal)

In [43]:
plan = move_group.go(wait=True)
move_group.stop()
move_group.clear_pose_targets()

ORIENTATION CONSTRAINT 넣어주기 

In [155]:

constraints = Constraints()

constraints.name = "constraints1"

oc1 = OrientationConstraint()

# input orientation constraints



constraints.orientation_constraints.append(oc1)
move_group.set_path_constraints(constraints)


pose_goal.position.x = 0.55
pose_goal.position.y = 0.4
pose_goal.position.z = 0.998 + 0.376

pose_goal.orientation.x = 0.768049136044
pose_goal.orientation.y = -0.00683168224924
pose_goal.orientation.z = 0.00497378503076
pose_goal.orientation.w = 0.640335157712
move_group.set_pose_target(pose_goal)

plan = move_group.go(wait=True)
move_group.stop()
move_group.clear_pose_targets()



In [124]:

import tf

tmp = tf.transformations.euler_from_quaternion([
    pose_goal.orientation.x,
    pose_goal.orientation.y,
    pose_goal.orientation.z,
    pose_goal.orientation.w])

[each*180/math.pi for each in tmp]

[100.36489387091633, -0.9390829294376395, -0.2363406321458759]

In [125]:
tf.transformations.quaternion_from_euler(30*math.pi/180,
                                         60*math.pi/180,
                                         90*math.pi/180)

array([-0.1830127,  0.5      ,  0.5      ,  0.6830127])

In [117]:
[p,q,r] = [each*math.pi/180 for each in [0., 60, 90.]]

x,y,z,w =tf.transformations.quaternion_from_euler(p,q,r)

NameError: name 'tf' is not defined

In [22]:
move_group.set_pose_target(pose_goal)

plan = move_group.go(wait=True)
move_group.stop()
move_group.clear_pose_targets()